# <center> Project One </center>
### <center> Dan Ortiz </center>
### <center> W205 Thursday 4:00 PM </center>
### <center> 9/24/2020 </center>

## Section One
This section of the notebook reflects the work performed for section one of the project. This entire section is in markdown.

### Initial Queries

- What's the size of this dataset? (i.e., how many trips)
    - The data set contains 983,648 trips and contains 983,646 unique trips with two duplicate entries. The bikeshare_trips table was used to calculate this number
    
    - The duplicate entries are:

    |Row|trip_id|start_station_name|start_station_id|end_dates|end_station_name|end_station_id|bike_number|zip_code|subscriber_type|
    |---|-------|------------------|----------------|---------|----------------|--------------|-----------|--------|---------------|
    |871529|258|2015-08-01 06:43:00 UTC|Temporary Transbay Terminal (Howard at Beale)| 55| 2015-08-01 06:47:00 UTC|Clay at Battery|41|517|94105|Subscriber|
    |871528|258|2015-08-01 06:43:00 UTC|Temporary Transbay Terminal (Howard at Beale)| 55| 2015-08-01 06:47:00 UTC|Clay at Battery|41|517|94105|Subscriber|
    |1058761|243|2016-01-12 17:03:00 UTC|2nd at Folsom|62|2016-01-12 17:08:00 UTC|Spear at Folsom|49|503|94105|Subscriber|
    |1058761|244|2016-01-12 17:03:00 UTC|2nd at Folsom|62|2016-01-12 17:07:00 UTC|Spear at Folsom|49|503|94105|Subscriber|
    
    - The SQL query to count the number of entries in the data set:
       ```sql
       #standardSQL
       SELECT COUNT(*)
       FROM (SELECT DISTINCT trip_id, 
       FROM bigquery-public-data.san_francisco.bikeshare_trips)
       ```
       <br>
     
    - Two duplicate trips were identified using the following query:
       ```sql
       #standardSQL
       SELECT a.*
       FROM bigquery-public-data.san_francisco.bikeshare_trips a
       JOIN 
          (SELECT start_date, 
                  bike_number, 
                  COUNT(*)
           FROM bigquery-public-data.san_francisco.bikeshare_trips 
           GROUP BY start_date, bike_number
           HAVING count(*) > 1 ) b
       ON a.start_date = b.start_date
       AND a.bike_number = b.bike_number
       ORDER BY a.start_date
       ```
       <br>


- What is the earliest start date and time and latest end date and time for a trip?
     - The bikeshare_trips table was used for the following queries.
         - Earliest Start Time: 2013-08-29 09:08:00 UTC
         - Earliest End Time:   2016-08-31 23:32:00 UTC
     
     - SQL Query used
     
     ```sql
     #standardSQL
     SELECT MIN(start_date) AS Min, MAX(start_date) AS Max
     FROM bigquery-public-data.san_francisco.bikeshare_trips
     ```
 
- How many bikes are there?
    - There are 700 unique bikes represented in the data set. The bikeshare_trips table was used to query this answer.
    
    ```sql
    SELECT COUNT(DISTINCT bike_number)
    FROM bigquery-public-data.san_francisco.bikeshare_trips
    ```


### Questions of your own

- Question 1: What is the shortest and longest trip taken by subscribers and customers? 
  * Answer:
   |subscriber_type|Min|Max|
   |---------------|---|---|
   |Customer|60|17270400|
   |Subscriber|60|1852590|

  * SQL query:
  
     ```sql
      #standardSQL
      SELECT subscriber_type, MIN(duration_sec) as Min, MAX(duration_sec) as Max
      FROM bigquery-public-data.san_francisco.bikeshare_trips
      GROUP BY subscriber_type
     ```

- Question 2: What is the name of the most and least popular destination stations?
  * Answer:
  
   Most Popular Destinations

   |Station|Number of Trips|
   |-------|---------------|
   |San Francisco Caltrain (Townsend at 4th)|92014|
   |San Francisco Caltrain 2 (330 Townsend)|58713|
   |Harry Bridges Plaza (Ferry Building)|50185|
   |Embarcadero at Sansome|46197|
   |2nd at Townsend|44145|
  
   Least Popular Destinations

   |Station|Number of Trips|
   |-------|---------------|
   |Cyril Magnin St at Ellis St|68|
   |5th S at E. San Salvador St|24|
   |San Jose Government Center |23|
   |Sequoia Hospital|14|
   |5th St at E. San Salvador St|1|
  * SQL query:
  
   ```sql
   #standardSQL
   SELECT end_station_name AS Station, COUNT(end_station_name) AS Trips
   FROM bigquery-public-data.san_francisco.bikeshare_trips
   GROUP BY end_station_name
   ORDER BY Trips DESC
   ```

- Question 3: What are the top 5 most popular routes 
  * Answer:
  
   |Row|start_station_name|end_station_name|Trips|
   |---|------------------|----------------|-----|
   |1|Harry Bridges Plaza (Ferry Building)|Emarcadero at Sansome|9150|
   |2|San Francisco Caltrain 2 (330 Townsent)|Townsend at 7th|8508|
   |3|2nd at Townsend|Harry Bridges Plaza(Ferry Building)|7620|
   |4|Harry Bridges Plaza(Ferry Building)|2nd at Townsend|6888|
   |5|Embarcadero at Sansome|Steuart at Market|6874|
   
  * SQL query:
  
     ```sql
     #standardSQL
     SELECT start_station_name, end_station_name, COUNT(trip_id) AS Trips
     FROM bigquery-public-data.san_francisco.bikeshare_trips
     GROUP BY start_station_name, end_station_name
     ORDER BY Trips DESC
     ```


### Bonus activity queries (optional - not graded - just this section is optional, all other sections are required)
The following queries use the dynamic data set, san_francisco_bikeshare and its associated tables, instead of the san_francisco data set.

- Top 5 popular station pairs in each region
  
  * Answer:
  
  * SQL Query:
  
      ```SQL
      #standardSQL
      SELECT si.region_id AS Region, 
             trip.start_station_id AS Start_Station, 
             trip.start_station_name Start_Station_Name, 
             trip.end_station_id AS End_Station, 
             trip.end_station_name AS End_STation_name,
             count(trip.start_station_id) AS Trips
      FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info` AS si
      JOIN `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` AS trip
      ON si.station_id = trip.start_station_id
      JOIN `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info` AS ei
      ON ei.station_id=trip.end_station_id
      WHERE si.region_id = ei.region_id
      GROUP BY si.region_id, trip.start_station_id, trip.start_station_name, trip.end_station_id, trip.end_station_name
      ORDER BY si.region_id, Trips DESC
      LIMIT 5
      ```
   
- Top 3 most popular regions(stations belong within 1 region)
  * Answer:
      Define popular region as destination.
     
      |Row|Region|Trips|
      |---|------|-----|
      |1|3|1,434,526|
      |2|12|147,799|
      |3|5|43,477|
     
  * SQL Query:

      ```SQL
      #standardSQL
      SELECT si.region_id AS Region, 
             count(trip.end_station_id) AS Trips
      FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info` AS si
      JOIN `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips` AS trip
      ON si.station_id = trip.end_station_id
      GROUP BY si.region_id
      ORDER BY Trips DESC
      LIMIT 3
      ```

- Total trips for each short station name in each region
  * Answer:
  * SQL Query:

- What are the top 10 used bikes in each of the top 3 region. these bikes could be in need of more frequent maintenance.
  * Answer:  
  * SQL Query:

## Part 2 - Querying data from the BigQuery CLI 

### Queries

1. Rerun the first 3 queries from Part 1 using bq command line tool (Paste your bq
   queries and results here, using properly formatted markdown):

  * What's the size of this dataset? (i.e., how many trips)

     * Query - Number of Reccords in the data set
         ```
         bq query --use_legacy_sql=false '
         > SELECT COUNT(*) AS count
         > FROM (SELECT DISTINCT trip_id, FROM bigquery-public-data.san_francisco.bikeshare_trips)
         ```
         <br>
         
     * Result - Number of Reccords in the data set
      
     | count  |
     |--------|
     | 983648 |

     * Query - Potential Duplicate Reccords
     
          ```
          bq query --use_legacy_sql=false 'SELECT a.* 
          > FROM bigquery-public-data.san_francisco.bikeshare_trips a 
          > JOIN (SELECT start_date, bike_number, COUNT(*) 
          >   FROM bigquery-public-data.san_francisco.bikeshare_trips 
          >   GROUP BY start_date, bike_number HAVING count(*) > 1 ) b 
          > ON a.start_date = b.start_date AND a.bike_number = b.bike_number ORDER BY a.start_date'
          ```
          <br>

     * Result - Potential Duplicate Reccords

     |trip_id|duration_sec|start_date|start_station_name|start_station_id|end_date|end_station_name|end_station_id|bike_number|zip_code|subscriber_type|
     |-------|------------|----------|------------------|----------------|--------|----------------|--------------|-----------|--------|---------------|
     |871529|258|2015-08-01 06:43:00|Temporary Transbay Terminal (Howard at Beale)|55|2015-08-01 06:47:00|Clay at Battery|41|517|94105|Subscriber|
     |871528|258|2015-08-01 06:43:00|Temporary Transbay Terminal (Howard at Beale)|55|2015-08-01 06:47:00|Clay at Battery|41|517|94105|Subscriber|
     |1058761|243|2016-01-12 17:03:00 |2nd at Folsom|62 |2016-01-12 17:08:00 |Spear at Folsom|49|503|94105|Subscriber|
     |1058782|244|2016-01-12 17:03:00 |2nd at Folsom|62 |2016-01-12 17:07:00 |Spear at Folsom|49|503|94105|Subscriber|


  * What is the earliest start time and latest end time for a trip?
  
     * Query
        ```
        bq query --use_legacy_sql=false '
        > SELECT MIN(start_date) AS Min, MAX(start_date) AS Max
        > FROM bigquery-public-data.san_francisco.bikeshare_trips'
        ```
        <br>
       
     * Result

     |         Min         |         Max         |
     |---------------------|---------------------|
     | 2013-08-29 09:08:00 | 2016-08-31 23:32:00 |


  * How many bikes are there?
 
     * Query
     
       ```
       bq query --use_legacy_sql=false '
       > SELECT COUNT(DISTINCT bike_number) AS count
       > FROM bigquery-public-data.san_francisco.bikeshare_trips'
      ```
     <br>
     
     * Result
     
      | count |
      |-------|
      |  700  |



#### Problem Statement

- Lyft Bay Wheels wants to increase ridership and wants to offer deals through its mobile app to do so (https://www.lyft.com/bikes/bay-wheels). Provide recommendations to Lyft, with supporting data, on what actions they should take to meet this goal.

* Current Deals from Lyft Bikes.
  * Annual Membership
     * \$149 per Year
     * Unlimited 45-Minute Classic Rides
     * Free upgrade to electric bike and discounted rate or \$0.15 per Min
     * Free parking at Bay Wheel Stations and \$2 parking at public bike racks
     * Access to entire network in San Francisco, East Bay and San Jose 
     
     <br>
     
  * Single Ride
     * \\$2 for 30 minutes and \$3 for each additional 15 minutes
     * E-Bikes for an additional \\$0.20 per minute for the first 30 min and \$3 for every 15 minutes after
     
     <br>  
     
  * Access Pass
     * \\$10 for unlimited 30 min rides for 24-hour period with \$3 for each additional 15 minutes
     * E-bikes only available in the Bay Wheels app that is linked to a valid Lyft account
     * E -Bikes encur additional charges
     
     <br>
     
  * Monthly Membership
     * Unlimited 45 min rides for 30 days
     * \$15 Per Month with no annual commitment
     * \$3 per additional 15 minutes
     * E-bikes additional \$0.15 per minute for the firts 45 minute
     * E-bikes \$2 for each additional 15 min
     
     <br>
  
  * Bike Share for All
     * Designed for income-eligible users
     * \$5 annual fee for first year
     * \$5 per month after the first year
     * Unlimited 60 min rides
     * E-bike additional \\$1 per trip and \$0.05 per minute
     
     <br>

  * Corporate Membership
     * Designed for corporations to provide subsidized bikes for employees
     
     <br>
  
  * Riding with Others
     * Designed for group rides, bulk access passes
     
     <br>


- Through this project, you will answer these questions: 

  * What are the 5 most popular trips that you would call "commuter trips"? 
  
  * What are your recommendations for offers (justify based on your findings)?

- Likewise to make your recommendations, you will need to do data exploration, cleaning or filtering dirty data, etc. to come up with the final queries that will give you the supporting data for your recommendations. You can make any recommendations regarding the offers, including, but not limited to: 
  * market offers differently to generate more revenue 
  * remove offers that are not working 
  * modify existing offers to generate more revenue
  * create new offers for hidden business opportunities you have found
  * etc.

### Project Questions
Identify the main questions you'll need to answer to make recommendations (list
below, add as many questions as you need).

- Question 1: How do we define what a commuter? What are the 5 most popular Commuter trips?

- Question 2: How do recreational trips differ between Customers and Subscribers?

- Question 3: What is the quantity of Subscriber Trips and Customer Trips?

- Question 4: When do our subscribers ride with us?

- Question 5: What stations stocks out the most?

### Answers

Answer at least 4 of the questions you identified above You can use either
BigQuery or the bq command line tool.  Paste your questions, queries and
answers below.

- Question 1:  How do we define what a commuter? What are the 5 most popular Commuter trips?

  * Answer:
  
     * Bikes must originate and destinate at different stations
     * Customer must be a subscriber
     
   |           start_station_name            | start_station_id |            end_station_name             | end_station_id | count |
   |-----------------------------------------|------------------|-----------------------------------------|----------------|-------|
   | San Francisco Caltrain 2 (330 Townsend) |               69 | Townsend at 7th                         |             65 |  8305 |
   | 2nd at Townsend                         |               61 | Harry Bridges Plaza (Ferry Building)    |             50 |  6931 |
   | Townsend at 7th                         |               65 | San Francisco Caltrain 2 (330 Townsend) |             69 |  6641 |
   | Harry Bridges Plaza (Ferry Building)    |               50 | 2nd at Townsend                         |             61 |  6332 |
   | Embarcadero at Sansome                  |               60 | Steuart at Market                       |             74 |  6200 |
  
  * SQL query:
   
   
        ```
        bq query --use_legacy_sql=false '
        > SELECT start_station_name,
        >        start_station_id,
        >        end_station_name,
        >        end_station_id,
        >        count(trip_id) As count
        > FROM bigquery-public-data.san_francisco.bikeshare_trips
        > WHERE start_station_id != end_station_id AND subscriber_type = "Subscriber"
        > GROUP BY start_station_name, start_station_id, end_station_name, end_station_id
        > ORDER BY count DESC
        > LIMIT 5'
        ``` 
       <br>

- Question 2: How do recreational trips differ between Customers and Subscribers?

  * Answer:
     
     * Recreational trips originate and destinate at the same station
     * Trips must be greater than 60 seconds (Filter out checkout check ins)
     
     * Results for order by customers


   |          start_station_name          | start_station_id | Subscriber | Customer |
   |--------------------------------------|------------------|------------|----------|
   | Embarcadero at Sansome               |               60 |        321 |     2545 |
   | Harry Bridges Plaza (Ferry Building) |               50 |        360 |     2004 |
   | The Embarcadero at Sansome St        |                6 |        239 |     1585 |
   | University and Emerson               |               35 |         71 |     1113 |
   | Central Ave at Fell St               |               70 |        161 |     1029 |
   
     * Results for order by Subscriber
     

   |  start_station_name   | start_station_id | Subscriber | Customer |
   |-----------------------|------------------|------------|----------|
   | Davis at Jackson      |               42 |        529 |      298 |
   | 3rd St at Townsend St |               66 |        521 |       56 |
   | 2nd at Townsend       |               61 |        451 |      444 |
   | Townsend at 7th       |               65 |        427 |      168 |
   | 2nd at South Park     |               64 |        387 |      123 |   
   
 
  
  * SQL queries:
  
     * Most popular recreational stations for Customers
     
     ```
     bq query --use_legacy_sql=false '
     > SELECT start_station_name,
     >        start_station_id,
     > SUM(CASE WHEN subscriber_type = "Subscriber" THEN 1 ELSE 0 END) AS Subscriber,
     > SUM(CASE WHEN subscriber_type = "Customer" THEN 1 ELSE 0 END) AS Customer
     > FROM bigquery-public-data.san_francisco.bikeshare_trips
     > WHERE start_station_id = end_station_id
     > GROUP BY start_station_name, start_station_id
     > ORDER BY Customer DESC
     > LIMIT 5'
     ```
     <br>
     
     * Most popular recreational stations for Subscribers
     
     ```
     bq query --use_legacy_sql=false '
     > SELECT start_station_name,
     >        start_station_id,
     > SUM(CASE WHEN subscriber_type = "Subscriber" THEN 1 ELSE 0 END) AS Subscriber,
     > SUM(CASE WHEN subscriber_type = "Customer" THEN 1 ELSE 0 END) AS Customer
     > FROM bigquery-public-data.san_francisco.bikeshare_trips
     > WHERE start_station_id = end_station_id
     > GROUP BY start_station_name, start_station_id
     > ORDER BY Subscriber DESC
     > LIMIT 5'
     ```
     <br>
     
- Question 3: What is the quantity of Subscriber Trips and Customer Trips?

  * Answer:

   | subscriber_type | trips  |
   |-----------------|--------|
   | Subscriber      | 846839 |
   | Customer        | 136809 |
   
  * SQL query:
   
       ```
       bq query --use_legacy_sql=false '
       > SELECT subscriber_type,
       > count(trip_id) As tri[s
       > FROM bigquery-public-data.san_francisco.bikeshare_trips 
       > GROUP BY subscriber_type
       > ORDER BY count DESC'
       ```
       <br>
  
- Question 4: When do our subscribers ride with us?

  * Answer:

   | Hour | Trips |
   |------|-------|
   |   23 |  1493 |
   |   22 |  2285 |
   |   21 |  2808 |
   |   20 |  3741 |
   |   19 |  5556 |
   |   18 |  8771 |
   |   17 | 11387 |
   |   16 | 12704 |
   |   15 | 12806 |
   |   14 | 12737 |
   |   13 | 12719 |
   |   12 | 12508 |
   |   11 | 11078 |
   |   10 |  8250 |
   |    9 |  6572 |
   |    8 |  5293 |
   |    7 |  2585 |
   |    6 |   945 |
   |    5 |   303 |
   |    4 |   160 |
   |    3 |   208 |
   |    2 |   413 |
   |    1 |   639 |
   |    0 |   848 |
  
  * SQL query:
  
     ```
     bq query --use_legacy_sql=false '
     > SELECT EXTRACT(HOUR FROM start_date) As Hour,
     >        COUNT(trip_id) AS Trips
     > FROM bigquery-public-data.san_francisco.bikeshare_trips
     > WHERE subscriber_type = "Customer"
     > GROUP BY Hour
     > ORDER BY Hour DESC'
     ```
     <br>
  
- Question 5: What day of the week do our subscribers ride?

  * Answer:
   | Day | Trips |
   |-----|-------|
   |   7 | 29244 |
   |   6 | 19929 |
   |   5 | 16612 |
   |   4 | 15237 |
   |   3 | 14737 |
   |   2 | 15142 |
   |   1 | 25908 |
  
  * SQL query:
     ```
     bq query --use_legacy_sql=false '
     > SELECT EXTRACT(DAYOFWEEK FROM start_date) As Day,
     >        COUNT(trip_id) AS Trips
     > FROM bigquery-public-data.san_francisco.bikeshare_trips
     > WHERE subscriber_type = "Customer"
     > GROUP BY Hour
     > ORDER BY Hour DESC'
     ```
     <br>

- Question 6:
  * Answer:
  * SQL query: